### 1) Import the Relevant Files & Libraries

In [1]:
import sib_main.sib_sub.sib_validation_main as svm
import sib_main.sib_sub.sib_validation_imports as svi

import copy
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, Point
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.multipoint import MultiPoint
from shapely.ops import nearest_points
from datetime import datetime

ground_truth = gpd.read_file("E:/SIB/package/data/model3a_GoodPasture/inputs/ground_truth/GT.shp")
model_det = gpd.read_file("E:/SIB/package/data/model3a_GoodPasture/inputs/detected_points/det.shp")
VHRsamples = gpd.read_file("E:/SIB/package/data/model3a_GoodPasture/inputs/vhr_samples/VHRValidation_samples.shp")

export_path = "E:/SIB/package/data/model3a_GoodPasture/outputs/"

### 2) Preprocess the Data

In [2]:
svi.confidence_filter(model_det)

90.0% of values above = 0.2104917


0.2104917

In [3]:
help(svi.multi_to_single)

Help on function multi_to_single in module sib_main.sib_sub.sib_validation_imports:

multi_to_single(gdf)
    This function creates POINT geometries from MULTIPOINT geometries.
    It does so by simply creating a centroid for each point and returning the gdf at the end.



In [4]:
# Change 'geometry' column from 'MULTIPOINT' to 'POINT'
ground_truth = svi.multi_to_single(ground_truth)
model_det = svi.multi_to_single(model_det)

In [5]:
help(svm.preprocessing)

Help on function preprocessing in module sib_main.sib_sub.sib_validation_main:

preprocessing(ground_truth_points, model_detected_points, conf_percentile=10)
    Preprocesses the data to a certain format:
    a) checks geometry type
    b) filters the confidence values by a chosen percentile
    c) verifies the correct crs
    d) establishes a unique ID for each GeoDataFrame
    e) renames the relevant columns to a standard format
    f) drops any unrelated columns and attributes



In [6]:
ground_truth, model_det = svm.preprocessing(ground_truth, model_det, conf_percentile=10)

Accepted Geometries: 'POINT'
Accepted Geometries: 'POINT'
90.0% of values above = 0.2104917


In [7]:
print("Number of Ground Truth Points: {}".format(len(ground_truth)))
print("Number of Model Detected Points: {}".format(len(model_det)))

Number of Ground Truth Points: 2151
Number of Model Detected Points: 2037


### 3) Find the maximum value of detected points within a 2m radius of a ground truth point

In [8]:
n_ = svi.n_max(ground_truth, model_det)

In [9]:
n_

4

### 4) Run the body of the script

In [10]:
startTime = datetime.now()

global_tp = []
global_fp = []

global_fn_df = svm.calculate_fn(ground_truth, model_det)
print("FALSE NEGATIVES GEODATAFRAME CONSTRUCTED\n")

for i in range(0, n_):

    # 1) CALCULATE NEAREST
    nearest_neighbour = svm.call_calculate_nearest(ground_truth, model_det)
    
    # 2) CALCUALTE DISTANCE
    nearest_neighbour = svm.calculate_distance(nearest_neighbour)
    
    # 3) CALCULATE TP
    nearest_neighbour, tp, tp_previous_pass = svm.calculate_tp(nearest_neighbour)
    tp_dict = tp.to_dict('list')
    try:
        global_tp.append(tp_dict)
    except AttributeError:
        print("No more True Positives to append")
        pass
 
    # 4) CALCULATE FP
    nearest_neighbour, fp = svm.calculate_fp(nearest_neighbour)
    fp_dict = fp.to_dict('list')
    try:
        global_fp.append(fp_dict)
    except AttributeError:
        print("No more False Positives to append")
        pass
         
    # 5) DROP INDICES
        # DROP TP INDICES
    model_det = svi.drop_indices(model_det, tp, outcome_value='true')
    
        # DROP FP INDICES
    model_det = svi.drop_indices(model_det, fp, outcome_value='false')
    
        # DROP GROUND TRUTH INDICES
    ground_truth = svi.drop_indices(ground_truth, tp, outcome_value='ground')
    
# 6) CREATE GEODATAFRAMES FOR TRUE POSITIVES AND FALSE POSITIVES
global_tp_df = svi.create_global_GeoDataFrame(global_tp)
global_fp_df = svi.create_global_GeoDataFrame(global_fp)
print("TRUE POSITIVEs GEODATAFRAME CONSTRUCTED\n")

# 7) CONSTRUCT THE COMPLETE FALSE POSITIVES DATAFRAME
fp2 = svi.extract_points_outside_2m(ground_truth, model_det)
global_fp_df = global_fp_df.append(fp2)
print("FALSE POSITIVES GEODATAFRAME CONSTRUCTED\n")

# 7) CALCULATE FN
global_fn_df = svm.calculate_fn(ground_truth, model_det)
print("FALSE NEGATIVES GEODATAFRAME CONSTRUCTED\n")

# 8) CREATE_VALIDATION_DATAFRAME
VHRsamples = svm.create_validation_dataframe(VHRsamples, global_tp_df, global_fp_df, global_fn_df)
print("VHRVALIDATION GEODATAFRAME CONSTRUCTED")

"""# 9) EXPORT GEODATAFRAME
svm.export_GeoDataFrame(export_path, VHRsamples, file_name='VHRSamples', extension='.shp')
svm.export_GeoDataFrame(export_path, global_tp_df, file_name='TP', extension='.shp')
svm.export_GeoDataFrame(export_path, global_fp_df, file_name='FP', extension='.shp')
svm.export_GeoDataFrame(export_path, global_fn_df, file_name='FN', extension='.shp')
svm.export_GeoDataFrame(export_path, ground_truth, file_name='ground_truth_OUTPUT', extension='.shp')
svm.export_GeoDataFrame(export_path, model_det, file_name='model_det_OUTPUT', extension='.shp')"""

print(datetime.now() - startTime)

FALSE NEGATIVES GEODATAFRAME CONSTRUCTED

Initial True Positives, Count: 1948
Initial False Positives, Count: 117
Length of 'model_det' after TP rows dropped: 255

Length of 'model_det' after FP rows dropped: 160

Length of 'Ground Truth points' after TP rows dropped: 203

Initial True Positives, Count: 23
There were no incorrect classifications (FP1) detected on this pass.
Length of 'model_det' after TP rows dropped: 152

Length of 'model_det' after FP rows dropped: 152

Length of 'Ground Truth points' after TP rows dropped: 180

Initial True Positives, Count: 0
There were no incorrect classifications (FP1) detected on this pass.
Length of 'model_det' after TP rows dropped: 152

Length of 'model_det' after FP rows dropped: 152

Length of 'Ground Truth points' after TP rows dropped: 180

Initial True Positives, Count: 0
There were no incorrect classifications (FP1) detected on this pass.
Length of 'model_det' after TP rows dropped: 152

Length of 'model_det' after FP rows dropped: 152


In [11]:
svm.export_GeoDataFrame(export_path, VHRsamples, file_name='VHRSamples', extension='.shp')
svm.export_GeoDataFrame(export_path, global_tp_df, file_name='TP', extension='.shp')
svm.export_GeoDataFrame(export_path, global_fp_df, file_name='FP', extension='.shp')
svm.export_GeoDataFrame(export_path, global_fn_df, file_name='FN', extension='.shp')
svm.export_GeoDataFrame(export_path, ground_truth, file_name='ground_truth_OUTPUT', extension='.shp')
svm.export_GeoDataFrame(export_path, model_det, file_name='model_det_OUTPUT', extension='.shp')

Exported to Local Drive
Exported to Local Drive
Exported to Local Drive
Exported to Local Drive
Exported to Local Drive
Exported to Local Drive
